In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.utils import Sequence
from sklearn.svm import SVC
from sklearn.metrics import roc_curve
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt
import joblib
import cv2 as cv


In [ ]:
#from google.colab import drive

# Mount Google Drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
img=cv.imread(r"D:\Engineering\Projects\Violence detection\Dataset_4 Blood_Detection_Dataset\Blood_Images\230.jpg")
cv.imshow("img",img)
cv.waitKey(0)

-1

In [3]:
# Constants
BINS = 32
BIN_WIDTH = 256 // BINS
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32

class BloodDetectionModel:
    def __init__(self):
        self.svm_classifier = SVC(probability=True)
        self.history = {'train_accuracy': [], 'train_loss': [], 'test_accuracy': [], 'test_loss': []}
        self.blood_histogram = None
        self.non_blood_histogram = None
        self.optimal_threshold = 0.5

    def create_3d_histogram(self, dataset):
        histogram = np.zeros((BINS, BINS, BINS))
        for images, _ in dataset:
            for img in images:
                img = img.numpy().astype(int)
                for pixel in img.reshape(-1, 3):
                    r, g, b = pixel // BIN_WIDTH
                    histogram[r, g, b] += 1
        histogram /= histogram.sum()
        return histogram

    def load_datasets(self, blood_dir, non_blood_dir):
        self.blood_dataset = self.load_dataset(blood_dir)
        self.non_blood_dataset = self.load_dataset(non_blood_dir)

        self.blood_histogram = self.create_3d_histogram(self.blood_dataset)
        self.non_blood_histogram = self.create_3d_histogram(self.non_blood_dataset)

    def load_dataset(self, directory):
        return image_dataset_from_directory(directory, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE, label_mode='int')

    def compute_blood_probability(self, image):
        h, w, _ = image.shape
        bpm = np.zeros((h, w))
        for i in range(h):
            for j in range(w):
                r, g, b = image[i, j] // BIN_WIDTH
                p_blood = self.blood_histogram[r, g, b]
                p_non_blood = self.non_blood_histogram[r, g, b]
                bpm[i, j] = p_blood / (p_blood + p_non_blood + 1e-7)
        return bpm

    def extract_features(self, bpm):
        binarized_bpm = (bpm > self.optimal_threshold).astype(int)
        labeled_bpm = label(binarized_bpm)
        regions = regionprops(labeled_bpm)
        features = [
            bpm.mean(),
            bpm.var(),
            len(regions),
            max([region.area for region in regions], default=0),
            sum([region.area for region in regions]),
        ]
        return features

    def prepare_training_data(self, dataset, label):
        X, y = [], []
        for images, labels in dataset:
            for img, _ in zip(images, labels):
                img = img.numpy().astype(int)
                bpm = self.compute_blood_probability(img)
                features = self.extract_features(bpm)
                X.append(features)
                y.append(label)
        return np.array(X), np.array(y)

    def train_with_generator(self, blood_generator, non_blood_generator, epochs=10):
        for epoch in range(epochs):
            for X_train, y_train in zip(blood_generator, non_blood_generator):
                self.svm_classifier.fit(X_train, y_train)

                train_accuracy = self.svm_classifier.score(X_train, y_train)
                self.history['train_accuracy'].append(train_accuracy)

                print(f"Epoch {epoch+1}/{epochs}:")
                print(f"  - Train Accuracy: {train_accuracy:.4f}")

        self.plot_training_history()

    def plot_training_history(self):
        plt.figure(figsize=(12, 5))

        # Plot accuracy
        plt.subplot(1, 2, 1)
        plt.plot(self.history['train_accuracy'], label='Train Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.title('Training Accuracy')
        plt.legend()

        plt.show()

    def determine_optimal_threshold(self, X_val, y_val):
        y_probs = self.svm_classifier.decision_function(X_val)
        fpr, tpr, thresholds = roc_curve(y_val, y_probs)
        optimal_idx = np.argmax(tpr - fpr)
        self.optimal_threshold = thresholds[optimal_idx]
        print(f"Optimal Threshold: {self.optimal_threshold}")

    def save_model(self, model_path):
        joblib.dump(self.svm_classifier, model_path)

    def load_model(self, model_path):
        self.svm_classifier = joblib.load(model_path)

    def predict_blood(self, image_path):
        test_image = tf.image.decode_image(tf.io.read_file(image_path)).numpy()
        bpm = self.compute_blood_probability(test_image)
        features = self.extract_features(bpm)
        prediction = self.svm_classifier.predict([features])
        return prediction, bpm

In [4]:
class CustomDataGenerator(Sequence):
    def __init__(self, dataset, histogram_blood, histogram_non_blood, batch_size=BATCH_SIZE, augment=False):
        self.dataset = dataset
        self.histogram_blood = histogram_blood
        self.histogram_non_blood = histogram_non_blood
        self.batch_size = batch_size
        self.augment = augment
        self.indices = np.arange(len(self.dataset))

    def __len__(self):
        return int(np.ceil(len(self.dataset) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size: (index + 1) * self.batch_size]
        X, y = [], []

        for idx in batch_indices:
            img, label = self.dataset[idx]
            img = img.numpy().astype(int)
            bpm = self.compute_blood_probability(img)
            features = self.extract_features(bpm)
            X.append(features)
            y.append(label)

        return np.array(X), np.array(y)

    def on_epoch_end(self):
        np.random.shuffle(self.indices)

    def compute_blood_probability(self, image):
        h, w, _ = image.shape
        bpm = np.zeros((h, w))
        for i in range(h):
            for j in range(w):
                r, g, b = image[i, j] // BIN_WIDTH
                p_blood = self.histogram_blood[r, g, b]
                p_non_blood = self.histogram_non_blood[r, g, b]
                bpm[i, j] = p_blood / (p_blood + p_non_blood + 1e-7)
        return bpm

    def extract_features(self, bpm):
        binarized_bpm = (bpm > 0.5).astype(int)
        labeled_bpm = label(binarized_bpm)
        regions = regionprops(labeled_bpm)
        features = [
            bpm.mean(),
            bpm.var(),
            len(regions),
            max([region.area for region in regions], default=0),
            sum([region.area for region in regions]),
        ]
        return features


In [6]:
import os
blood_dir = r"D:\Engineering\Projects\Violence detection\Dataset_4 Blood_Detection_Dataset\Blood_Images"
non_blood_dir = r"D:\Engineering\Projects\Violence detection\Dataset_4 Blood_Detection_Dataset\Non_Blood_Images"
# Check contents of blood_dir
print(os.listdir(blood_dir))

# Check contents of non_blood_dir
print(os.listdir(non_blood_dir))

['10.jpg', '100.jpg', '101.jpg', '102.jpg', '103.jpg', '104.jpg', '105.jpg', '106.jpg', '107.jpg', '108.jpg', '109.jpg', '110.jpg', '111.jpg', '112.jpg', '113.jpg', '114.jpg', '115.jpg', '116.jpg', '117.jpg', '118.jpg', '119.jpg', '12.jpg', '120.jpg', '121.jpg', '122.jpg', '123.jpg', '124.jpg', '125.jpg', '126.jpg', '127.jpg', '128.jpg', '129.jpg', '130.jpg', '131.jpg', '132.jpg', '133.jpg', '134.jpg', '135.jpg', '136.jpg', '137.jpg', '138.jpg', '139.jpg', '140.jpg', '141.jpg', '142.jpg', '143.jpg', '144.jpg', '145.jpg', '146.jpg', '147.jpg', '148.jpg', '149.jpg', '150.jpg', '151.jpg', '152.jpg', '153.jpg', '154.jpg', '155.jpg', '156.jpg', '157.jpg', '158.jpg', '159.jpg', '160.jpg', '161.jpg', '162.jpg', '163.jpg', '164.jpg', '165.jpg', '166.jpg', '167.jpg', '168.jpg', '169.jpg', '170.jpeg', '171.jpg', '172.jpg', '173.jpg', '174.jpg', '175.jpg', '176.jpg', '177.jpg', '178.jpg', '179.jpg', '180.jpg', '181.jpg', '182.jpg', '183.jpg', '184.jpg', '185.jpg', '186.jpg', '187.jpg', '188.jpg',

In [8]:
if __name__ == "__main__":
   # Load the datasets
    blood_dir = r"D:\Engineering\Projects\Violence detection\Dataset_4 Blood_Detection_Dataset\Blood_Images"
    non_blood_dir = r"D:\Engineering\Projects\Violence detection\Dataset_4 Blood_Detection_Dataset\Non_Blood_Images"

    model = BloodDetectionModel()
    model.load_datasets(blood_dir, non_blood_dir)


    # Create the data generators
    blood_generator = CustomDataGenerator(
        dataset=list(model.blood_dataset.unbatch().as_numpy_iterator()),
        histogram_blood=model.blood_histogram,
        histogram_non_blood=model.non_blood_histogram,
        batch_size=BATCH_SIZE
    )

    non_blood_generator = CustomDataGenerator(
        dataset=list(model.non_blood_dataset.unbatch().as_numpy_iterator()),
        histogram_blood=model.blood_histogram,
        histogram_non_blood=model.non_blood_histogram,
        batch_size=BATCH_SIZE
    )

    # Prepare validation data
    X_blood_val, y_blood_val = model.prepare_training_data(model.blood_dataset, label=1)
    X_non_blood_val, y_non_blood_val = model.prepare_training_data(model.non_blood_dataset, label=0)
    X_val = np.concatenate([X_blood_val, X_non_blood_val], axis=0)
    y_val = np.concatenate([y_blood_val, y_non_blood_val], axis=0)

    # Train the model using the generators
    model.train_with_generator(blood_generator, non_blood_generator, epochs=10)

    # Determine the optimal threshold using validation data
    model.determine_optimal_threshold(X_val, y_val)


    model_path = r"Model/blood_detection_svm.h5"

    blood_detection_model = BloodDetectionModel()
    blood_detection_model.load_datasets(blood_dir, non_blood_dir)
    blood_detection_model.train(epochs=10)
    blood_detection_model.save_model(model_path)
    # Save the trained model
    model.save_model(model_path)



Found 0 files belonging to 0 classes.


ValueError: No images found in directory D:\Engineering\Projects\Violence detection\Dataset_4 Blood_Detection_Dataset\Blood_Images. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

In [ ]:
    test_image_path = 'path/to/test_image.jpg'
    prediction, bpm = model.predict_blood(test_image_path)

  # Plot the test image
    plt.figure(figsize=(8, 8))
    test_image = tf.image.decode_image(tf.io.read_file(test_image_path))
    plt.subplot(1, 2, 1)
    plt.imshow(test_image)
    plt.title('Test Image')
    plt.axis('off')

    # Plot the BPM
    plt.subplot(1, 2, 2)
    plt.imshow(bpm, cmap='hot', interpolation='nearest')
    plt.colorbar()
    plt.title('Blood Probability Map')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    print(f"Prediction: {'Blood' if prediction[0] == 1 else 'Non-Blood'}")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Example image path (replace with your image path)
image_path = '/content/drive/MyDrive/Blood_Detection_Dataset/Blood_Images/30.jpg'

# Load image
img = mpimg.imread(image_path)

# Display image
plt.imshow(img)
plt.axis('off')  # Optional: Turn off axis ticks and labels
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Blood_Detection_Dataset/Blood_Images/30.jpg'